In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
%matplotlib inline

In [2]:
master = pd.read_csv('data/Master.csv', parse_dates=['debut', 'finalGame'])
# Currently not caring about pitchers

In [3]:
master.dtypes

playerID                object
birthYear              float64
birthMonth             float64
birthDay               float64
birthCountry            object
birthState              object
birthCity               object
deathYear              float64
deathMonth             float64
deathDay               float64
deathCountry            object
deathState              object
deathCity               object
nameFirst               object
nameLast                object
nameGiven               object
weight                 float64
height                 float64
bats                    object
throws                  object
debut           datetime64[ns]
finalGame       datetime64[ns]
retroID                 object
bbrefID                 object
dtype: object

In [4]:
def birth_state_count(state):
    print(master[master['birthState'] == state].size)

birth_state_count('RI')
birth_state_count('FL')

1848
11256


In [5]:
bbmaster = master[['playerID', 'nameFirst', 'nameLast', 'nameGiven']]
bbmaster.head()

,playerID,nameFirst,nameLast,nameGiven
0,aardsda01,David,Aardsma,David Allan
1,aaronha01,Hank,Aaron,Henry Louis
2,aaronto01,Tommie,Aaron,Tommie Lee
3,aasedo01,Don,Aase,Donald William
4,abadan01,Andy,Abad,Fausto Andres


In [6]:
batting = pd.read_csv('data/Batting.csv')
batting.columns

Index(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'G', 'AB', 'R', 'H',
       '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH',
       'SF', 'GIDP'],
      dtype='object')

In [7]:
batting.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,...,13,8,1,4,0,NaN,NaN,NaN,NaN,NaN
2,allisar01,1871,1,CL1,NaN,29,137,28,40,4,...,19,3,1,2,5,NaN,NaN,NaN,NaN,NaN
3,allisdo01,1871,1,WS3,NaN,27,133,28,44,10,...,27,1,1,0,2,NaN,NaN,NaN,NaN,NaN
4,ansonca01,1871,1,RC1,NaN,25,120,29,39,11,...,16,6,2,2,1,NaN,NaN,NaN,NaN,NaN


In [8]:
master_batting = pd.merge(master, batting)

In [9]:
batting.columns

Index(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'G', 'AB', 'R', 'H',
       '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH',
       'SF', 'GIDP'],
      dtype='object')

In [10]:
bats = batting[['playerID', 'yearID','AB', 'H', 'BB', 'HBP', 'SF']]
bats.head()
print(len(bats))
bbdata2 = pd.merge(bbmaster, bats)
bbdata2.head()

99846


,playerID,nameFirst,nameLast,nameGiven,yearID,AB,H,BB,HBP,SF
0,aardsda01,David,Aardsma,David Allan,2004,0,0,0,0,0
1,aardsda01,David,Aardsma,David Allan,2006,2,0,0,0,0
2,aardsda01,David,Aardsma,David Allan,2007,0,0,0,0,0
3,aardsda01,David,Aardsma,David Allan,2008,1,0,0,0,0
4,aardsda01,David,Aardsma,David Allan,2009,0,0,0,0,0


In [11]:
salaries = pd.read_csv('data/Salaries.csv')
salaries.columns

Index(['yearID', 'teamID', 'lgID', 'playerID', 'salary'], dtype='object')

In [12]:
salaries = salaries[['yearID', 'playerID', 'salary']]
print(len(salaries))
salaries.head()
len(salaries[salaries['salary'] == 0])

24758


2

In [13]:
bbdata_sal = pd.merge(bbdata2, salaries)
print(len(bbdata_sal))
bbdata_sal.head()

26470


,playerID,nameFirst,nameLast,nameGiven,yearID,AB,H,BB,HBP,SF,salary
0,aardsda01,David,Aardsma,David Allan,2004,0,0,0,0,0,300000
1,aardsda01,David,Aardsma,David Allan,2007,0,0,0,0,0,387500
2,aardsda01,David,Aardsma,David Allan,2008,1,0,0,0,0,403250
3,aardsda01,David,Aardsma,David Allan,2009,0,0,0,0,0,419000
4,aardsda01,David,Aardsma,David Allan,2010,0,0,0,0,0,2750000


In [14]:
# Drop all AB values of 0 or null (average at-bats in MLB is 30/year)
bbdata_sal = bbdata_sal.drop(bbdata_sal.index[(bbdata_sal['AB'] < 15) | \
                                              (bbdata_sal['H']  < 30) | \
                                              (pd.isnull(bbdata_sal['AB']))])

In [15]:
bbdata_sal['OBP'] = bbdata_sal.apply(lambda row: (row['H'] + row['BB'] + row['HBP']) * 100 / \
                                     (row['AB'] + row['BB'] + row['HBP'] + row['SF']), axis=1)

In [16]:
bbdata_sal.head()

,playerID,nameFirst,nameLast,nameGiven,yearID,AB,H,BB,HBP,SF,salary,OBP
14,abbotje01,Jeff,Abbott,Jeffrey William,1998,244,68,9,0,5,175000,29.844961
16,abbotje01,Jeff,Abbott,Jeffrey William,2000,215,59,21,2,1,255000,34.309623
29,abbotku01,Kurt,Abbott,Kurt Thomas,1994,345,86,16,5,2,109000,29.076087
30,abbotku01,Kurt,Abbott,Kurt Thomas,1995,420,107,36,5,5,119000,31.759657
31,abbotku01,Kurt,Abbott,Kurt Thomas,1996,320,81,22,3,0,250000,30.724638


In [17]:
bbdata_sal.sort(columns=['OBP'], ascending=False).head()

,playerID,nameFirst,nameLast,nameGiven,yearID,AB,H,BB,HBP,SF,salary,OBP
2367,bondsba01,Barry,Bonds,Barry Lamar,2004,373,135,232,9,3,18000000,60.940032
2365,bondsba01,Barry,Bonds,Barry Lamar,2002,403,149,198,9,2,15000000,58.169935
2366,bondsba01,Barry,Bonds,Barry Lamar,2003,390,133,148,10,2,15500000,52.909091
2364,bondsba01,Barry,Bonds,Barry Lamar,2001,476,156,177,9,2,10300000,51.506024
19501,ramirma02,Manny,Ramirez,Manuel Aristides,2008,187,74,35,3,4,18929923,48.908297


In [18]:
bbdata_sal = bbdata_sal.drop(bbdata_sal.index[bbdata_sal['salary'] <= 0])
pass

In [19]:
bbdata_sal[bbdata_sal['playerID'] == 'jamesdi01']

,playerID,nameFirst,nameLast,nameGiven,yearID,AB,H,BB,HBP,SF,salary,OBP
11605,jamesdi01,Dion,James,Dion,1987,494,154,70,2,3,120000,39.718805
11606,jamesdi01,Dion,James,Dion,1988,386,99,58,1,2,360000,35.346756
11607,jamesdi01,Dion,James,Dion,1989,170,44,25,1,1,400000,35.532995
11608,jamesdi01,Dion,James,Dion,1989,245,75,24,0,0,400000,36.802974
11609,jamesdi01,Dion,James,Dion,1990,248,68,27,1,1,650000,34.657040
11610,jamesdi01,Dion,James,Dion,1992,145,38,22,1,2,387500,35.882353
11612,jamesdi01,Dion,James,Dion,1995,209,60,20,0,2,350000,34.632035


In [20]:
bbdata_sal['OBP_sal_10k'] = bbdata_sal.apply(lambda row: row['OBP'] * 10000 / row['salary'], axis=1)
bbdata_sal.sort(columns='OBP_sal_10k', ascending=False).head()

,playerID,nameFirst,nameLast,nameGiven,yearID,AB,H,BB,HBP,SF,salary,OBP,OBP_sal_10k
12168,jonestr01,Tracy,Jones,Tracy Donald,1986,86,30,9,0,1,60000,40.625000,6.770833
12947,krukjo01,John,Kruk,John Martin,1986,278,86,45,0,2,60000,40.307692,6.717949
5375,danieka01,Kal,Daniels,Kalvoski,1986,181,58,22,2,1,60000,39.805825,6.634304
21661,seitzke01,Kevin,Seitzer,Kevin Lee,1987,641,207,80,2,1,62500,39.917127,6.386740
15159,mazzile01,Lee,Mazzilli,Lee Louis,1987,124,38,21,0,3,62500,39.864865,6.378378


In [21]:
appearances = pd.read_csv('data/Appearances.csv')
appearances.columns

Index(['yearID', 'teamID', 'lgID', 'playerID', 'G_all', 'GS', 'G_batting',
       'G_defense', 'G_p', 'G_c', 'G_1b', 'G_2b', 'G_3b', 'G_ss', 'G_lf',
       'G_cf', 'G_rf', 'G_of', 'G_dh', 'G_ph', 'G_pr'],
      dtype='object')

In [22]:
appearances.pop('lgID')
appearances.pop('teamID')
appearances.head()

,yearID,playerID,G_all,GS,G_batting,G_defense,G_p,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,G_of,G_dh,G_ph,G_pr
0,1871,barnero01,31,NaN,31,31,0,0,0,16,0,15,0,0,0,0,NaN,NaN,NaN
1,1871,barrofr01,18,NaN,18,18,0,0,0,1,0,0,13,0,4,17,NaN,NaN,NaN
2,1871,birdsda01,29,NaN,29,29,0,7,0,0,0,0,0,0,27,27,NaN,NaN,NaN
3,1871,conefr01,19,NaN,19,19,0,0,0,0,0,0,18,0,1,18,NaN,NaN,NaN
4,1871,gouldch01,31,NaN,31,31,0,0,30,0,0,0,0,0,1,1,NaN,NaN,NaN


In [48]:
mb_data = pd.merge(appearances, bbdata_sal)
mb_data.head()

,yearID,playerID,G_all,GS,G_batting,G_defense,G_p,G_c,G_1b,G_2b,...,nameLast,nameGiven,AB,H,BB,HBP,SF,salary,OBP,OBP_sal_10k
0,1985,benedbr01,70,67,70,70,0,70,0,0,...,Benedict,Bruce Edwin,208,42,22,1,2,545000,27.896996,0.511871
1,1985,ceronri01,96,76,96,90,0,90,0,0,...,Cerone,Richard Aldo,282,61,29,1,4,625000,28.797468,0.460759
2,1985,chambch01,101,27,101,39,0,0,39,0,...,Chambliss,Carroll Christopher,170,40,18,0,1,800000,30.687831,0.383598
3,1985,harpete01,138,124,138,131,0,0,0,0,...,Harper,Terry Joe,492,130,44,3,2,250000,32.717190,1.308688
4,1985,hornebo01,130,125,130,127,0,0,87,0,...,Horner,James Robert,483,129,50,1,6,1500000,33.333333,0.222222


## Rules of Baseball

Baseball has 9 defensive positions:
* pitcher (who does not hit, and are therefore excluded from this analysis; however, I include designated hitter/pinch hitters)
* catcher
* 1st base
* 2nd base
* short stop
* 3rd base
* left field
* center field
* right field


Because `G_of` is just a count of whether a player was `G_lf`, `G_cf`, or `G_rf`, it does not consititue an actual spot on the roster.

In [134]:
ROSTER=['G_c', 'G_1b', 'G_2b','G_3b', 'G_ss', 'G_lf', 'G_cf', 'G_rf', 'G_dh', 'G_ph']

In [135]:
def mb_players(year, pos):
    return mb_data[(mb_data['yearID'] == year) & (mb_data[pos] > 5)][:10].sort(columns='OBP_sal_10k', ascending=False)

I have set a minimum of 5 games in the position to be considered for the roster.

In [127]:
def mb_roster(year):
    mbr_uf = []
    for pos in ROSTER:
        mbr_uf.append(mb_players(year, pos))
        
    ret = []
    num_pos = len(mbr_uf)
    overlap = []
    for m, pos in enumerate(mbr_uf):
        ret.append(pos.iloc[0])
        try:
            for n in range(num_pos):
                if m != n:
                    overlap.append(n)
                    mbr_uf[n].iloc[0] == pos.iloc[0]
            overlap = []
        except:
            if mbr_uf[overlap[-1]].iloc[1].OBP_sal_10k > mbr_uf[m].iloc[1].OBP_sal_10k:
                ret[m] = mbr_uf[overlap[-1]].iloc[1]
            else:
                mbr_uf[m].iloc[1]
            overlap = []
    return pd.DataFrame(ret)

In [141]:
def mb_roster(year):
    mbr_uf = []
    for pos in ROSTER:
        mbr_uf.append(mb_players(year, pos))
        
    ret = []
    no_overlap = []
    player_list = []
    for pos in mbr_uf:
        for n in range(len(pos)):
            if pos.iloc[n].playerID not in player_list:
                no_overlap.append(n)
        player_list.append(pos.iloc[no_overlap[0]].playerID)
        ret.append(pos.iloc[no_overlap[0]])
        no_overlap = []
                       
    return pd.DataFrame(ret)

In [143]:
roster2001 = mb_roster(2001)
roster2001

,yearID,playerID,G_all,GS,G_batting,G_defense,G_p,G_c,G_1b,G_2b,...,nameLast,nameGiven,AB,H,BB,HBP,SF,salary,OBP,OBP_sal_10k
6225,2001,wootesh01,79,61,79,47,0,25,21,0,...,Wooten,William Shawn,221,69,5,3,3,200500,33.189655,1.655344
6219,2001,joynewa01,53,38,53,39,0,0,39,0,...,Joyner,Wallace Keith,148,36,13,0,0,200000,30.434783,1.521739
6214,2001,eckstda01,153,146,153,138,0,0,0,14,...,Eckstein,David Mark,582,166,43,21,2,200000,35.493827,1.774691
6245,2001,helmswe01,100,51,100,92,0,0,77,0,...,Helms,Wesley Ray,216,48,21,1,1,210000,29.288703,1.394700
6217,2001,gilbe01,104,73,104,78,0,0,18,21,...,Gil,Romar Benjamin,260,77,14,0,2,350000,32.971014,0.942029
6228,2001,delluda01,115,40,115,58,0,0,0,0,...,Dellucci,David Michael,217,60,22,2,0,325000,34.854772,1.072455
6275,2001,richach01,136,125,136,111,0,0,18,0,...,Richard,Christopher Robert,483,128,45,8,4,232500,33.518519,1.441657
6226,2001,bautida01,100,42,100,61,0,0,0,0,...,Bautista,Daniel,222,67,14,1,0,675000,34.599156,0.512580
6231,2001,durazer01,92,34,92,40,0,0,38,0,...,Durazo,Erubiel,175,47,28,2,2,260000,37.198068,1.430695
6220,2001,kennead01,137,127,137,131,0,0,0,131,...,Kennedy,Adam Thomas,478,129,27,11,9,280000,31.809524,1.136054


In [81]:
G_2b = mb_players(1996, 'G_2b')
G_2b.columns

Index(['yearID', 'playerID', 'G_all', 'GS', 'G_batting', 'G_defense', 'G_p',
       'G_c', 'G_1b', 'G_2b', 'G_3b', 'G_ss', 'G_lf', 'G_cf', 'G_rf', 'G_of',
       'G_dh', 'G_ph', 'G_pr', 'nameFirst', 'nameLast', 'nameGiven', 'AB', 'H',
       'BB', 'HBP', 'SF', 'salary', 'OBP', 'OBP_sal_10k'],
      dtype='object')

In [82]:
G_3b = mb_players(1996, 'G_3b')

In [83]:
try:
    G_3b[G_2b['playerID']]
except:
    print("No overlaps")

No overlaps


In [89]:
G_3b.iloc[0]

yearID              1996
playerID       houstty01
G_all                 46
GS                    28
G_batting             46
G_defense             39
G_p                    0
G_c                   27
G_1b                   1
G_2b                   2
G_3b                   9
G_ss                   0
G_lf                   0
G_cf                   0
G_rf                   0
G_of                   0
G_dh                   0
G_ph                   9
G_pr                   2
nameFirst          Tyler
nameLast         Houston
nameGiven      Tyler Sam
AB                   115
H                     39
BB                     8
HBP                    0
SF                     0
salary            109000
OBP             38.21138
OBP_sal_10k     3.505631
Name: 3907, dtype: object